# PAD - Praca domowa 6
## Jan Solarz s26342

In [192]:
import pandas as pd
#data = pd.read_csv('C:/Users/Jan Solarz/Downloads/SMSSpamCollection.txt', sep=" ", header=None)
df = pd.read_csv('C:/Users/Jan Solarz/Downloads/SMSSpamCollection.txt', sep="\t", names = ["spam","text"])
#data.columns = ["a", "b", "c", "etc."]

Dokonujemy klasyfikacji sms-ów ze względu na to, czy są spamem czy nie.

In [193]:
df

,spam,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [194]:
df['spam'].value_counts()

ham     4825
spam     747
Name: spam, dtype: int64

In [195]:
 t = df['spam'].tolist()
for i in range(0,len(t)):
    if t[i] == 'spam':
        t[i] = 1
    else:
        t[i] = 0

In [196]:
df['spam'] = t

In [197]:
from sklearn.feature_extraction.text import TfidfVectorizer
def clean_text(x):
    x = x.lower()
    x = x.replace('.','')
    x = x.replace(',','')
    x = x.replace(':','')
    x = x.replace(';','')
    x = x.replace('!','')
    x = x.replace('?','')
    x = x.replace('<br>','')
    x = x.replace('<br />','')
    x = ' '.join([word for word in x.split() if word.isalpha()])
    return x

df['text'] = df['text'].apply(clean_text)

tfidf = TfidfVectorizer() #by zredukować rozmiar macierzy uwzględniamy tylko słowa, ktore pojawiły się w conajmniej 10 dokumentach

inputs= tfidf.fit_transform(df['text'])
target = df['spam']

In [198]:
inputs =inputs.todense()

- Jako dane wejścia wykorzystujemy wagi TF-IDF
- Dodajemy nformacje o sentymencie smsa do modelu klasyfikacyjnego


In [199]:
a = pd.DataFrame(inputs, columns = tfidf.get_feature_names()) #reprezentacja w formie ramki danych
a

,aa,aah,aaniye,aaooooright,aathilove,aathiwhere,ab,abbey,abdomen,abeg,...,zebra,zed,zeros,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [201]:
import re
from spacy.tokenizer import Tokenizer
import spacy
from collections import Counter
import spacy.attrs
import en_core_web_sm
nlp = en_core_web_sm.load()

Dodajemy informacę o proporcji czasowników („VERB”), rzeczowników („NOUN”) i przymiotników („ADJ”)

In [202]:
n0 = []
v0 = []
a0 = []
for i in range(0,len(df['text'])):
    docx = nlp(df['text'][i])
    dist = len(docx)
    nouns = [token.text for token in docx if token.is_stop != True and token.is_punct != True and token.pos_ == 'NOUN']
    verbs = [token.text for token in docx if token.is_stop != True and token.is_punct != True and token.pos_ == 'VERB']
    adj = [token.text for token in docx if token.is_stop != True and token.is_punct != True and token.pos_ == 'ADJ']
    if dist != 0:     
        n0.append(round(len(nouns)/dist,2))
        v0.append(round(len(verbs)/dist,2))
        a0.append(round(len(adj)/dist,2))
    else:
        n0.append(round(len(nouns)/1,2))
        v0.append(round(len(verbs)/1,2))
        a0.append(round(len(adj)/1,2))

In [203]:
a['Verb'] = v0
a['Noun'] = n0
a['Adjective'] = a0

In [204]:
a

,aa,aah,aaniye,aaooooright,aathilove,aathiwhere,ab,abbey,abdomen,abeg,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,Verb,Noun,Adjective
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.20,0.15
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.17,0.50,0.00
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.19,0.24,0.14
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.09,0.09
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.33,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.09,0.32,0.05
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.38,0.12,0.00
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.33,0.11
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12,0.12,0.08


In [205]:
#Dokonujemy walidacji danych
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split(inputs,target, test_size=0.25)


In [206]:
#Reprezentacja z użyciem funkcji ważącej TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
from sklearn.metrics import *
import numpy as np

Wykorzystujemy Regresje logistyczną, KNN i Naiwny klasyfikator Bayesowski do klasyfikacji maili. 

In [207]:
#Logistic regression
from sklearn.linear_model import LogisticRegression
model_loggit = LogisticRegression()
model_loggit.fit(train_input,train_target)
predictions = model_loggit.predict(test_input)

print("Positive sentiment precision:",precision_score(test_target,predictions))
print("Positive sentiment recall:",recall_score(test_target,predictions))
print("Negative sentiment precision:",precision_score(test_target,predictions,pos_label = 0))
print("Negative sentiment recall:",recall_score(test_target,predictions,pos_label = 0))
print("Classification accuracy:",accuracy_score(test_target,predictions))

Positive sentiment precision: 0.9929078014184397
Positive sentiment recall: 0.7106598984771574
Negative sentiment precision: 0.9544728434504792
Negative sentiment recall: 0.9991638795986622
Classification accuracy: 0.9583632447954056


In [208]:
#KNN classifier
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(train_input,train_target)
preds_knn = model_knn.predict(test_input)

In [209]:
prec_knn = precision_score(test_target,preds_knn)
recall_knn = recall_score(test_target,preds_knn)
accuracy_knn = accuracy_score(test_target,preds_knn)
roc_knn = roc_auc_score(test_target,preds_knn)

print("Positive sentiment precision:",precision_score(test_target,preds_knn,pos_label = 1))
print("Positive sentiment recall:",recall_knn)
print("Negative sentiment precision:",precision_score(test_target,preds_knn,pos_label = 0))
print("Negative sentiment recall:",recall_score(test_target,preds_knn,pos_label = 0))
print("Classification accuracy:",accuracy_knn)

Positive sentiment precision: 1.0
Positive sentiment recall: 0.49746192893401014
Negative sentiment precision: 0.9235521235521236
Negative sentiment recall: 1.0
Classification accuracy: 0.9289303661162958


In [210]:
#NB classifier
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()
#test_input = test_input.toarray()
#train_input = train_input.toarray()
model_nb.fit(train_input,train_target)
preds_NB = model_nb.predict(test_input)

In [211]:
prec_NB = precision_score(test_target,preds_NB)
recall_NB = recall_score(test_target,preds_NB)
accuracy_NB = accuracy_score(test_target,preds_NB)
roc_NB = roc_auc_score(test_target,preds_NB)
print("Positive sentiment precision:",prec_NB)
print("Positive sentiment recall:",recall_NB)
print("Negative sentiment precision:",precision_score(test_target,preds_NB,pos_label = 0))
print("Negative sentiment recall:",recall_score(test_target,preds_NB,pos_label = 0))
print("Classification accuracy:",accuracy_NB)

Positive sentiment precision: 0.531055900621118
Positive sentiment recall: 0.868020304568528
Negative sentiment precision: 0.9757236227824463
Negative sentiment recall: 0.8737458193979933
Classification accuracy: 0.8729361091170137


- Najlepiej poradziły sobie modele Logit osiągając blisko 96% skuteczności
- Recall sentymentu pozytywnego w KNN wyniosła 49%, natomiast negatywnego 100%
- Wszystkie algorytmy działają na zadowalającym poziomie